## 0. Imports

In [53]:
import json
from substrateinterface import ss58_decode, ss58_encode
from collections import defaultdict

## 1. Parse Data Used For ASTR Reward Distribution & On-Chain Data


In [2]:
# There seem to be bugs in this data so we'll compare it to data read off-chain
with open('sdn-balance-snapshot-753857.json') as wrong_snapshot:
    sdn_balance_wrong_snapshot = { x['address'] : int(x['balance']) for x in json.load(wrong_snapshot) }

In [3]:
with open('sdn-ksm-crowdloan-reward.json') as crowdloan_reward:
    sdn_balance_crwd_reward = { x['account_id'] : int(x['amount']) for x in json.load(crowdloan_reward) }

In [45]:
balance_key_map = lambda x: x['account'][0]
balance_value_map = lambda x: int(x['data']['data']['free'].replace(",", ""))

with open('sdn_balance_from_chain.json') as on_chain_balance:
    on_chain_balance_snapshot = { balance_key_map(x) : balance_value_map(x) for x in json.load(on_chain_balance) }

## 2. Determine Who Incorrectly (Didn't) Receive Bonus Reward

In [8]:
# How much decrease in reward we allow
reward_buffer = 10 ** 17

is_eligible = lambda init, final: final  >= (init - reward_buffer) 

In [20]:
got_reward = set()
didnt_get_reward = set()

# We start with Crowdloan rewards since that's the basis on which bonus is distributed
for account, crowdloan_reward in sdn_balance_crwd_reward.items():
        
    got_bonus_paid = is_eligible(crowdloan_reward, sdn_balance_wrong_snapshot[account])
    should_have_got_bonus = is_eligible(crowdloan_reward, on_chain_balance_snapshot[account])
    
    if got_bonus_paid is True and should_have_got_bonus is False:
        got_reward.add(account)
        
    if got_bonus_paid is False and should_have_got_bonus is True:
        didnt_get_reward.add(account)

In [21]:
len(got_reward), len(didnt_get_reward)

(1, 734)

In [22]:
# Read lockdrop participant accounts and remove them from missing accounts
with open('plasm_to_duration.json') as plm_to_duration_file:
    plasm_addresses = set(json.load(plm_to_duration_file).keys())

In [23]:
filtered_got_reward = got_reward - plasm_addresses
filtered_didnt_get_reward = didnt_get_reward - plasm_addresses

In [24]:
print(f"After removing PLASM addresses (which received bonus since they were LD participants), we get {len(filtered_got_reward)} account(s) which received reward when they shouldn't have and {len(filtered_didnt_get_reward)} account(s) which didn't receive reward when they should have.")

After removing PLASM addresses (which received bonus since they were LD participants), we get 1 account(s) which received reward when they shouldn't have and 733 account(s) which didn't receive reward when they should have.


## 3. Consolidate DOT Contributions

In [29]:
with open('polkadot-crowdloan.json') as dot_crowdloan_file:
    dot_crowdloan_raw = json.load(dot_crowdloan_file)
    dot_crowdloan_raw.reverse()

In [52]:
dot_key_mapper = lambda x: ss58_encode(ss58_decode(x['who']), 5)
dot_value_mapper = lambda x: int(x['contributing'])

In [54]:
dot_contribution_count = defaultdict(int)

for dot_lock_event in dot_crowdloan_raw:
    astr_account = dot_key_mapper(dot_lock_event)
    dot_amount = dot_value_mapper(dot_lock_event)
    
    dot_contribution_count[astr_account] = dot_contribution_count[astr_account] + dot_amount

In [58]:
dot_reward_multiplier = 101610752585225

multiplier_decimals = 10 ** 12
dot_decimals = 10 ** 10

astr_map = lambda dot: dot * dot_reward_multiplier / multiplier_decimals / dot_decimals 

base_astar_reward = { k: astr_map(v) for k, v in dot_contribution_count.items() }

In [68]:
# Sanity check
(dot_contribution_count['WFmkoRtBmM6zrH6mECrVhCjLAT8THNLinfikpgXbBUsjxX1'] / 10 ** 10, 
base_astar_reward['WFmkoRtBmM6zrH6mECrVhCjLAT8THNLinfikpgXbBUsjxX1'])

(5.0, 508.053762926125)

## 4. Process Accounts With Missing Rewards

In [85]:
not_astar_contributor = set()

missing_bonus = dict()

for account in filtered_didnt_get_reward:
    if account not in base_astar_reward.keys():
        # Didn't contribute to Astar crowdloan
        not_astar_contributor.add(account)
        continue
        
    missing_bonus[account] = base_astar_reward[account] / 10

In [86]:
len(not_astar_contributor)

527

In [87]:
len(missing_bonus)

206

In [91]:
with open('missing_10_percent_bonus.json', 'w') as f:
    json.dump(missing_bonus, f)